In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd

In [2]:
class Network(nn.Module):
    def __init__(self, p1, p2):
        super().__init__()
        self.features = nn.Sequential(
            nn.Linear(31, 31, bias= True),
            nn.ReLU(),
            nn.Linear(31, 31, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p1),
            nn.Linear(31, 30, bias = True),
            nn.ReLU(),
            nn.Linear(30, 30, bias= True),
            nn.ReLU(),
            nn.Linear(30, 30, bias= True),
            nn.ReLU(),
            nn.Dropout(p= p2),
            nn.Linear(30, 25, bias= True),
            nn.ReLU(),
            nn.Linear(25, 20, bias= True),
            nn.ReLU(),
            nn.Linear(20, 15, bias= True),
            nn.ReLU(),
            nn.Linear(15, 10, bias= True),
            nn.ReLU(),
            nn.Linear(10, 5, bias = True),
            nn.ReLU()
        )
        self.classify = nn.Sequential(
            nn.Linear(5, 1, bias= True),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.classify(self.features(x))

In [3]:
class Data(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = torch.tensor(x.values, dtype= torch.float32)
        self.y = torch.tensor(y.values, dtype= torch.float32)
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, index):
        x, y = self.x[index], self.y[index]
        return x, y

In [4]:
xtrain, ytrain, xtest, ytest = pd.read_csv('./X_train.csv', header= None, index_col = None), pd.read_csv('./y_train.csv', header = None, index_col = None), pd.read_csv('./X_test.csv', header = None, index_col = None), pd.read_csv('./y_test.csv', header = None, index_col = None)
train_data = Data(x= xtrain, y= ytrain)
test_data = Data(x= xtest, y= ytest)
train_loader = DataLoader(train_data, batch_size=10000, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 10000)

In [5]:
model = Network(0.3, 0.6)
model.to('cuda')

Network(
  (features): Sequential(
    (0): Linear(in_features=31, out_features=31, bias=True)
    (1): ReLU()
    (2): Linear(in_features=31, out_features=31, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=31, out_features=30, bias=True)
    (6): ReLU()
    (7): Linear(in_features=30, out_features=30, bias=True)
    (8): ReLU()
    (9): Linear(in_features=30, out_features=30, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.6, inplace=False)
    (12): Linear(in_features=30, out_features=25, bias=True)
    (13): ReLU()
    (14): Linear(in_features=25, out_features=20, bias=True)
    (15): ReLU()
    (16): Linear(in_features=20, out_features=15, bias=True)
    (17): ReLU()
    (18): Linear(in_features=15, out_features=10, bias=True)
    (19): ReLU()
    (20): Linear(in_features=10, out_features=5, bias=True)
    (21): ReLU()
  )
  (classify): Sequential(
    (0): Linear(in_features=5, out_features=1, bias=True)
    (1): Sigmoid()
  )
)

In [6]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay= 0.001)

In [7]:
for epoch in range(100):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to('cuda'), target.to('cuda')
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'loss - {running_loss}')

loss - 34.31029969453812
loss - 33.8978071808815
loss - 33.8809620141983
loss - 33.8882954120636
loss - 33.88553899526596
loss - 33.890077352523804
loss - 33.88453710079193
loss - 33.88616579771042
loss - 33.89145475625992
loss - 33.89017617702484
loss - 33.897412836551666
loss - 33.89099943637848
loss - 33.89171123504639
loss - 33.89288252592087
loss - 33.8935741186142
loss - 33.889671981334686
loss - 33.883439779281616
loss - 33.88964486122131
loss - 33.89181327819824
loss - 33.89153218269348
loss - 33.891111731529236
loss - 33.892656207084656
loss - 33.89246243238449
loss - 33.88726818561554
loss - 33.895561039447784
loss - 33.88588231801987
loss - 33.893262922763824
loss - 33.88752102851868
loss - 33.88687229156494
loss - 33.8894938826561
loss - 33.88780742883682
loss - 33.89153045415878
loss - 33.89276272058487
loss - 33.88433641195297
loss - 33.88443332910538
loss - 33.88840639591217
loss - 33.886161267757416
loss - 33.89842128753662
loss - 33.88866329193115
loss - 33.88438504934

In [8]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = torch.round(model(input))
        all_preds.extend(output.to('cpu'))
        all_labels.extend(target.to('cpu'))

from sklearn.metrics import accuracy_score
print(accuracy_score(all_preds, all_labels))


0.6156712407294616
